In [1]:
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
from sql_metadata import Parser
import json
import psycopg2
import psycopg2.extras

In [4]:
# with open('table_n_cols.json', 'r') as f:
#     table_n_cols = json.load(f)
# table_n_cols[0]

{'table_name': 'actor', 'column_name': 'actor_id', 'data_type': 'integer'}

In [13]:
table_n_cols_agg = {}
for entry in table_n_cols:
    table_name = entry['table_name']
    column_name = entry['column_name']
    if table_name in table_n_cols_agg:
        table_n_cols_agg[table_name].append(column_name)
    else:
        table_n_cols_agg[table_name] = [column_name]

In [14]:
agg_df = pd.DataFrame(list(table_n_cols_agg.items()), columns=['table_name', 'columns'])
agg_df.head()

,table_name,columns
0,actor,"[actor_id, first_name, last_name, last_update]"
1,actor_info,"[actor_id, first_name, last_name, film_info]"
2,customer_list,"[id, name, address, zip code, phone, city, cou..."
3,film_list,"[fid, title, description, category, price, len..."
4,nicer_but_slower_film_list,"[fid, title, description, category, price, len..."


In [23]:
query = """
SELECT category.name, AVG(film.rental_rate) AS average_rental_rate
FROM category
JOIN film_category ON category.category_id = film_category.category_id
JOIN film ON film_category.film_id = film.film_id
GROUP BY category.name;
"""

In [24]:
parsed_query = Parser(query)
tables = parsed_query.tables
columns = parsed_query.columns

In [28]:
columns

['category.name',
 'film.rental_rate',
 'category.category_id',
 'film_category.category_id',
 'film_category.film_id',
 'film.film_id']

In [12]:
def extract_table_column_mapping(query, table_n_cols_agg):
    # Parse the query to get tables and columns
    parsed_query = Parser(query)
    tables = parsed_query.tables
    columns = parsed_query.columns

    table_column_mapping = {}
    # Initialize a dictionary to hold the table-column mappings
    for table in tables:
        if table in table_n_cols_agg:
            table_columns = table_n_cols_agg[table]
            filtered_columns = set()
            for col in columns:
                # Check if the column is prefixed with a table name or alias
                parts = col.split(".")
                column_name = parts[-1]  # Get the actual column name, regardless of prefix
                if column_name in table_columns:
                    filtered_columns.add(column_name)
            if filtered_columns:
                table_column_mapping[table] = list(filtered_columns)
    
    return table_column_mapping

In [41]:
query = """
SELECT category.name, AVG(film.rental_rate) AS average_rental_rate
FROM category
JOIN film_category ON category.category_id = film_category.category_id
JOIN film ON film_category.film_id = film.film_id
GROUP BY category.name;
"""

In [42]:
test = extract_table_column_mapping(query, table_n_cols_agg)
test

{'category': ['name', 'category_id'],
 'film_category': ['film_id', 'category_id'],
 'film': ['film_id', 'rental_rate']}

In [24]:
conn = psycopg2.connect("dbname=dvdrental host='192.168.16.16' user='ags14' port=5430 password='bitnine123!'")
conn.autocommit = True
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
cursor.execute(f"""
                SET search_path TO ag_catalog;
                """)

In [99]:
query = """
    SELECT
        table_name,
        column_name,
        data_type
    FROM
        information_schema.columns
    ORDER BY
        table_name, ordinal_position;
    """
cursor.execute(query)
cursor.fetchall()

[RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'start_id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'start_id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'end_id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'end_id'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),
              ('column_name', 'properties'),
              ('data_type', 'USER-DEFINED')]),
 RealDictRow([('table_name', '_ag_label_edge'),


In [45]:
# data = pd.read_csv('SQL_querylogs_dataset - dvdrental.csv')
# data.head()

,query,action,detail,ref,쿼리\n작동 여부,쿼리 적합성,쿼리 처리 소요시간(ms)\n(최적화 전),쿼리 처리 소요시간(ms)\n(최적화 후),비고,Unnamed: 9,Unnamed: 10,최적화 전,최적화 후
0,"SELECT customer_id, COUNT(rental_id) AS total_...",고객별 총 대여 횟수 조회,rental 테이블에서 고객별로 rental_id를 카운트,rental,O,NaN,NaN,-,NaN,NaN,쿼리 처리 소요시간 평균(ms),NaN,NaN
1,"SELECT inventory.film_id, AVG(return_date - re...",각 영화의 평균 대여 기간 계산,rental 테이블과 inventory 테이블을 조인하여 영화별로 대여 기간의 평균...,"rental, inventory",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
2,"SELECT customer_id, genre, MAX(genre_count) AS...",각 고객별로 가장 많이 대여한 장르 조회,RANK() 함수로 각 고객별로 장르 대여 횟수에 따라 순위를 매김. PARTIT...,"rental, inventory, film_category, category",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
3,SELECT EXTRACT(YEAR FROM rental_date) AS renta...,모든 고객의 대여 기록이 있는 월별 대여 횟수,rental 테이블에서 연도와 월을 추출하여 그 기간 동안의 총 대여 횟수를 집계,rental,O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN
4,"SELECT film.film_id, title, SUM(amount) AS tot...",각 영화에 대한 총 수익 조회\n\n,"payment, rental, inventory, film 테이블을 조인하여 각 영...","payment, rental, inventory, film",O,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN


In [7]:
sample_audit = pd.read_csv('sample_agaudit-2024-04-13-16-00-00.csv', header=None)
sample_audit.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,1,SELECT,2024-04-13 16:02:12 KST,3/330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
1,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,2,SELECT,2024-04-13 16:02:12 KST,3/331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
2,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,3,SELECT,2024-04-13 16:02:12 KST,3/332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
3,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,4,SELECT,2024-04-13 16:02:12 KST,3/333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement
4,2024-04-13 16:04:46.112 KST,ags14,dvdrental,40580,192.168.16.16:38676,661a2df4.9e84,5,SELECT,2024-04-13 16:02:12 KST,3/334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,READ,NaN,statement


In [8]:
def trim(query):
    return query.replace('"'," ").replace("\n"," ").strip()

In [9]:
queries = [trim(i) for i in sample_audit[13]]
queries[:5]

['SELECT customer_id, COUNT(rental_id) AS total_rentals FROM rental GROUP BY customer_id;',
 'SELECT inventory.film_id, AVG(return_date - rental_date) AS average_rental_duration FROM rental JOIN inventory ON rental.inventory_id = inventory.inventory_id GROUP BY inventory.film_id;',
 'WITH GenreCounts AS (   SELECT      rental.customer_id,      category.name AS genre,      COUNT(*) AS genre_count,     MAX(COUNT(*)) OVER (PARTITION BY rental.customer_id) AS max_genre_count   FROM rental   JOIN inventory ON rental.inventory_id = inventory.inventory_id   JOIN film_category ON inventory.film_id = film_category.film_id   JOIN category ON film_category.category_id = category.category_id   GROUP BY rental.customer_id, category.name ) SELECT    customer_id,    genre,    genre_count AS max_genre_count FROM GenreCounts WHERE genre_count = max_genre_count;',
 'SELECT EXTRACT(YEAR FROM rental_date) AS rental_year, EXTRACT(MONTH FROM rental_date) AS rental_month, COUNT(rental_id) AS total_rentals FR

In [15]:
cooc_table_col = []
for i in queries:
    tables_cols = extract_table_column_mapping(i, table_n_cols_agg)
    cooc_table_col.append(tables_cols)
cooc_table_col[:5]

[{'rental': ['rental_id', 'customer_id']},
 {'rental': ['return_date', 'inventory_id', 'rental_date'],
  'inventory': ['inventory_id', 'film_id']},
 {'rental': ['inventory_id', 'customer_id'],
  'inventory': ['inventory_id', 'film_id'],
  'film_category': ['film_id', 'category_id'],
  'category': ['name', 'category_id']},
 {'rental': ['rental_date', 'rental_id']},
 {'payment': ['rental_id', 'amount'],
  'rental': ['inventory_id', 'rental_id'],
  'inventory': ['inventory_id', 'film_id'],
  'film': ['title', 'film_id']}]

In [ ]:
# with open('cooc_table_col.json', 'w') as f:
#     json.load(cooc_table_col, f)

In [3]:
with open('cooc_table_col.json', 'r') as f:
    cooc_table_col = json.load(f)

In [ ]:
def update_edge_count(table, column):
    cursor.execute(f"""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {{ table_name:'{table}' }})-[r]-(v2:column {{ column_name:'{column}' }})
                  SET r.count = r.count + 1
                  RETURN r.count 
                  $$) as (count agtype);
                  """)
    updated_count = int(cursor.fetchall()[0]['count'])
    print(f"edge count between table '{table}' and column '{column}' has been updated from {updated_count-1} to {updated_count}")

In [28]:
for i in cooc_table_col:
    for table in i.keys():
        cols = i[table]
        for col in cols:
            update_edge_count(table, col)

edge count between table 'rental' and column 'customer_id' has been updated from 4 to 5
edge count between table 'rental' and column 'rental_id' has been updated from 2 to 3
edge count between table 'rental' and column 'return_date' has been updated from 2 to 3
edge count between table 'rental' and column 'inventory_id' has been updated from 2 to 3
edge count between table 'rental' and column 'rental_date' has been updated from 2 to 3
edge count between table 'inventory' and column 'film_id' has been updated from 2 to 3
edge count between table 'inventory' and column 'inventory_id' has been updated from 0 to 1
edge count between table 'rental' and column 'customer_id' has been updated from 5 to 6
edge count between table 'rental' and column 'inventory_id' has been updated from 3 to 4
edge count between table 'inventory' and column 'film_id' has been updated from 3 to 4
edge count between table 'inventory' and column 'inventory_id' has been updated from 1 to 2
edge count between table '

In [21]:
def update_edge_count(table, column):
    cursor.execute(f"""
                  SELECT * FROM cypher('graph_v2',$$
                  MATCH (v1:table {{ table_name:'{table}' }})-[r]-(v2:column {{ column_name:'{column}' }})
                  SET r.count = r.count + 1
                  RETURN r.count 
                  $$) as (count agtype);
                  """)
    updated_count = int(cursor.fetchall()[0]['count'])
    print(f"edge count between table '{table}' and column '{column}' has been updated from {updated_count-1} to {updated_count}")